## REDO

It's fairly intuitively clear that there is some sort of close relationship between computing, programs and mathematical proofs, but what is the relationship exactly?  The answer from type theory is the "Curry-Howard correspondence" between proof systems and type systems.  

In Coda, the situation is much simpler. Rather than "type" being a foundational concept with no definition, in Coda, a type is merely any data T which is both idempotent (T:T:X = T:X) and distributive ( T:X Y = (T:X) (T:Y) ).  In Coda, everything is data and the only valid operation on data is to apply a definition.  A proof and a computation are exactly the same thing: a sequence of data D1=D2=...=Dn.  

Here, let's echo the famous elaborate proof from Principia Mathematica that 1+1=2.

In [1]:
#
#    First, let's just compute 1+1 with the natural number type "n".
#
sum n : 1 1

(n:2)

In [2]:
#
#    This result depends on a use of Python integers because the type n uses Python to convert from 
#    unicode strings to integers and to do the addition.  
#
#    We could remove this dependency by using peano natural numbers where a number n is represented by a sequence of 
#    n atoms.
#
sum peano : 1 1 

((Sum:peano) peano:1 1)

In [3]:
#
#....To show terminal display instead of showing a list of tokens...
#
term peano : sum peano : 1 1 

((Term:peano) peano:((Sum:peano) peano:1 1))

Let's discuss the basis for claiming that this is a **proof** that 1+1 = 2.  

The main point is that the only thing Coda can do is apply definitions to data.  For example, there is a definition `rev` which is **defined** to be reversing the order of a finite sequence.  So, by definition, 

`(rev : 1 2 3 4 5) = 5 4 3 2 1`

`rev` happens to be a built-in definition, defined in Python code, but it is the same for definitions defined by users in the Coda language. **data** is a very simple Python class and the `eval` method clearly does nothing but apply definitions from the global CONTEXT object containing built-in and user provided definitions.  Once can read each of the ~50 built-in definitions like "rev" and verify that each one is a partial function and verify the mechanism that insures that the domains of the definitions are disjoint.  This could be done once and for all.

For each particular proof, one can list the individual steps as well.  Here are the steps for the peano integer version of 1+1=2.  You will see that there are more than 40 individual steps.  The first steps include data in curly braces, because this is actually a proof that the coda 

* `{term peano : sum peano : 1 1}:`

is equal to the coda

* `2`

If there was an extra blank space between `sum` and `peano` for instance, the proof would be very slightly different.  
 

In [4]:
step : term peano : sum peano : 1 1 

[ 0] (({term peano }:):({ sum peano : 1 1}:))
[ 1] (({term peano}:):({sum peano : 1 1}:))
[ 2] (({term}:) ({peano}:):(({sum peano }:):({ 1 1}:)))
[ 3] ({(Term:A) A : B} peano:(({sum peano}:):({1 1}:)))
[ 4] (({(Term:A) A } peano:(({sum}:) ({peano}:):({1}:) ({1}:))):({ B} peano:(({sum}:) ({peano}:):({1}:) ({1}:))))
[ 5] (({(Term:A) A} peano:({(Sum:A)  A : B} peano:1 1)):({B} peano:({(Sum:A)  A : B} peano:1 1)))
[ 6] (({(Term:A)} peano:(({(Sum:A)  A } peano:1 1):({ B} peano:1 1))) ({A} peano:(({(Sum:A)  A } peano:1 1):({ B} peano:1 1))):(({(Sum:A)  A } peano:1 1):({ B} peano:1 1)))
[ 7] (({Term:A} peano:(({(Sum:A)  A} peano:1 1):({B} peano:1 1))) peano:(({(Sum:A)  A} peano:1 1):({B} peano:1 1)))
[ 8] ((({Term} peano:(({(Sum:A)} peano:1 1) ({ A} peano:1 1):1 1)):({A} peano:(({(Sum:A)} peano:1 1) ({ A} peano:1 1):1 1))) peano:(({(Sum:A)} peano:1 1) ({ A} peano:1 1):1 1))
[ 9] ((Term:peano) peano:(({Sum:A} peano:1 1) ({A} peano:1 1):1 1))
[10] ((Term:peano) peano:((({Sum} peano:1 1):({A} pe

In [5]:
#
#   This computation implies that (term peano : sum peano : 1 1) is equal to 2 in the current context. 
#   You can verify this directly via, for instance,...
#
(term peano : sum peano : 1 1 ) = 2

(= ((Term:peano) peano:((Sum:peano) peano:1 1)):2)

Notice that if A=B is true (or false), it remains so independent of any definitions added to the context where A=B is originally computed. This is certainly a necessary property for what we think of as a proof. 

On the other hand, if A=B is undecided data, it may become true or false depending on future definitions.  For example, 

`x? = 45`

is undecided.  It is neither true nor false. If, however x? gets defined to be 45 later, it becomes true...

In [6]:
x? = 45

(= (?:x):45)

In [7]:
let x? : 45

In [8]:
x? = 45

This is an interesting and potentially useful feature.  Addition of peano numbers should be commutative, but how is that proven?  First, we need to define what commutative means.

In [9]:
#
#   As you can see, Commutative already has a definition.
#
help : Commutative

code:
    Commutative
module:
    Algebra
summary:
    Commutative
description:
path:
    /Users/youssef/coda/co/Algebra.co
source code:
    def Commutative : { B : X? Y? = B : Y? X? }
demos:
    1. Commutative : pass
    2. Commutative : sum n
    3. Commutative : prod n


This means that data D is Commutative if `Commutative : D` is true, i.e. if `Commutative : D=()`.  Thus, we should have 

`Commutative : sum peano = ()` 

Let's try...

In [10]:
Commutative : sum peano 

((Sum:peano) peano:((= (?:X) (?:Y):sum peano):(?:Y) (?:X)))

Notice that we don't get true and we don't get false, but instead, we get some rather complicated looking undecided expression.  This expression can be thought of as "the obstruction" to what we want to prove.  If we can get it to disappear, we have the result that we think is actually true. 

Of course, we still have an obstruction for a good reason: The commutativity and associativity of peano numbers comes from the associativity and commutativity of concatenating sequences of some any single coda, e.g. 

(◎ ◎) (◎ ◎ ◎)  = (◎ ◎ ◎) (◎ ◎)

We can take this fact to be true as part of our foundational understanding of finite sequences, but this fact is relevant only because I happen to know how peano integers are implemented.  In fact, this suggests defining SameAs A to be the category whose objects are data consisting of zero or more repetitions of the sequence A.  Then the objects of SameAs ◎, for instance, would be finite sequences of the atom ◎, morphisms would be data F where F:X is in SameAs ◎ if X is in SameAs ◎.  SameAs ◎ has a commutative and associative addition and multiplication.  This suggests implementing Peano in SameAs ◎ and relying on proven properties of SameAs ◎ to prove things about the peano integers. This actually also suggests generalization to SameAs A where A is more than a single atom.  

We claim that it is a fact that if A and B are sequences of a single coda (say ◎), then A B = B A.  This suggests defining something like 

def SameAs : {apif {B=A} A : B} 

so SameAs a : a b a b z a = a a a 

SameAs A is then a type for any 

This is meant to be a baby example of an approach towards more interesting mathematical questions.  

is not (yet) a useful theorem in the system.  



In [11]:
def SameAs : {apif {B=A} A : B} 

In [12]:
SameAs a : a b a a b a (:) z z 

a a a a

In [13]:
def SameAtom : SameAs (:)

In [14]:
SameAtom : a b (:) (:) z z z 

◎ ◎

In [15]:
(SameAtom : SameAtom : a b (:) (:) z z z ) = (SameAtom : a b (:) (:) z z z )